In [1]:
import pandas as pd
import requests
import json
from fredapi import Fred
import numpy as np
import requests
import os
from tqdm import tqdm

from bs4 import BeautifulSoup
import json
import matplotlib.pyplot as plt
import quandl
import re
import seaborn as sns

# BLS Data

## 1. LOCAL AREA UNEMPLOYMENT STATISTICS 
- [Link](https://download.bls.gov/pub/time.series/la/)

In [2]:
BLS_url = 'https://download.bls.gov/pub/time.series/la/'
files = 'la.data.0.CurrentU15-19'

In [3]:
r = requests.get(BLS_url+files)
r

<Response [200]>

In [4]:
area_data = [row.split('\t') for row in r.text.split('\n')]
area_header = area_data[0]
area_body = area_data[1:]

areas_ls = []
for row in area_body:
    a_dict= {}
    for col_num, val in enumerate(row):
        a_dict[area_header[col_num]] = val
    areas_ls.append(a_dict)
    
df_area = pd.DataFrame(areas_ls).dropna()

In [6]:
df_area.head(3)

,series_id,year,period,value,footnote_codes\r
0,LAUBS060000000000003,2015,M01,7.0,\r
1,LAUBS060000000000003,2015,M02,6.7,\r
2,LAUBS060000000000003,2015,M03,6.6,\r


In [7]:
df_area.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2163707 entries, 0 to 2163706
Data columns (total 5 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   series_id                       object
 1   year                            object
 2   period                          object
 3          value                    object
                 object
dtypes: object(5)
memory usage: 99.0+ MB


## 1.2 Time series
- https://download.bls.gov/pub/time.series/la/

In [8]:
url = 'https://download.bls.gov/pub/time.series/la/la.data.2.AllStatesU'
r = requests.get(url)
r

<Response [200]>

In [9]:
area_data = [row.split('\t') for row in r.text.split('\n')]
area_header = area_data[0]
area_body = area_data[1:]

In [10]:
areas_ls = []
for row in area_body:
    a_dict= {}
    for col_num, val in enumerate(row):
        a_dict[area_header[col_num]] = val
    areas_ls.append(a_dict)
    
df_area = pd.DataFrame(areas_ls).dropna()

In [11]:
df_area.head(3)

,series_id,year,period,value,footnote_codes\r
0,LAUST010000000000003,1976,M01,7.5,\r
1,LAUST010000000000003,1976,M02,7.7,\r
2,LAUST010000000000003,1976,M03,6.3,\r


In [12]:
df_area.columns = [x.strip() for x in df_area.columns]
df_area.head(2)

,series_id,year,period,value,footnote_codes
0,LAUST010000000000003,1976,M01,7.5,\r
1,LAUST010000000000003,1976,M02,7.7,\r


In [13]:
df_area.series_id = [x.strip() for x in df_area.series_id]
df_area.value = [x.strip() for x in df_area.value]

In [14]:
url = 'https://bitbucket-students.deakin.edu.au/users/sunnyse/repos/companyandngo/raw/rcode/Statewise_Data.csv?at=f3a66771a58783e61a4465533e0cdb41d295d6f6'

id_df = pd.read_csv(url, encoding='iso-8859-1' #'utf-8'
                   )
series_map = id_df.iloc[:, [1,3]].set_index('seriesID').to_dict()['State_Name']

In [15]:
for key, value in series_map.items():
    print(key,value)
    break

LAUST010000000000003 Alabama


In [16]:
url = 'https://raw.githubusercontent.com/fernrees/everify_diffusion/cdecd83eda4c0bcdf152cca620a980fe6daa29a9/data/originals/bls_data/bls_codes.csv'
id_df = pd.read_csv(url)
id_df.head(2)

,code,state
0,LAUST010000000000003,Alabama
1,LAUST020000000000003,Alaska


In [17]:
series_map = id_df.set_index('code').to_dict()['state']

In [18]:
df_area.series_id = df_area.series_id.replace(series_map).dropna()
df_area.head(2)

,series_id,year,period,value,footnote_codes
0,Alabama,1976,M01,7.5,\r
1,Alabama,1976,M02,7.7,\r


In [19]:
state_list = id_df.state.to_list()
mask = df_area.series_id.isin(state_list)
df_area = df_area.loc[mask]
df_area.head(2)

,series_id,year,period,value,footnote_codes
0,Alabama,1976,M01,7.5,\r
1,Alabama,1976,M02,7.7,\r


In [20]:
df_area.value = df_area.value.astype(float)
df_area.head(2)

,series_id,year,period,value,footnote_codes
0,Alabama,1976,M01,7.5,\r
1,Alabama,1976,M02,7.7,\r
